In [1]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [66]:
import utils
texts = utils.load_contexts('../corpora/ad-nouns-contexts-100k', 'горшок')
for text in texts[:3]:
    print(text)
print(len(texts))
#texts = texts[:1000]

['час', 'время', 'инцидент', 'хватить', 'некоторый', 'легенда', 'говориться', 'золото', 'находиться', 'конец', 'радуга', 'глубина', '22', 'фут', 'почва', 'становиться']
['гнить', 'нераскрытый', 'тайна', 'коридор', 'бояться', 'находить', 'черепок', 'глиняный', 'настоящий', 'череп', 'скелет', 'смочь', 'выбираться']
['орхидея', 'совершенно', 'очаровывать', 'становиться', 'любимый', 'цветок', 'рекомендовать', 'поддон', 'класть', 'гравий', 'немного', 'камень', 'сверху', 'ставить', 'орхидея']
34188


In [41]:
from gensim import corpora
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus[0], corpus[1])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)] [(16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1)]


In [58]:
#from gensim.models.ldamulticore import LdaMulticore as LdaModel
from gensim.models import LdaModel
print(len(corpus))
lda = LdaModel(corpus, id2word=dictionary, num_topics=6, passes=3, alpha='auto')

34188


In [59]:
lda.print_topics()

[(0,
  '0.014*ребенок + 0.013*год + 0.009*2222 + 0.009*мочь + 0.005*сидеть + 0.005*ходить + 0.005*время + 0.005*бог + 0.005*день + 0.005*становиться'),
 (1,
  '0.006*мед + 0.004*всякий + 0.004*голова + 0.003*император + 0.003*медведь + 0.003*вода + 0.003*глиняный + 0.003*песня + 0.003*прекрасный + 0.003*находить'),
 (2,
  '0.017*растение + 0.011*цветок + 0.011*земля + 0.009*22 + 0.009*цветочный + 0.006*2 + 0.006*окно + 0.006*место + 0.004*подоконник + 0.004*размер'),
 (3,
  '0.011*глиняный + 0.010*стоять + 0.007*цветок + 0.007*стол + 0.005*пол + 0.005*большой + 0.005*стена + 0.004*поставлять + 0.004*огонь + 0.004*кровать'),
 (4,
  '0.010*2 + 0.007*голова + 0.006*рука + 0.005*конь + 0.005*река + 0.004*крестьянин + 0.004*кот + 0.004*кошка + 0.004*котел + 0.003*месяц'),
 (5,
  '0.010*ночной + 0.008*рука + 0.008*знать + 0.006*говорить + 0.006*пойти + 0.005*мочь + 0.004*понимать + 0.004*глаз + 0.004*ничто + 0.004*называть')]

In [6]:
import rl_wsd_labeled
senses, contexts = rl_wsd_labeled.get_contexts(rl_wsd_labeled.contexts_filename('nouns', 'RuTenTen', 'горшок'))
print(senses)
print(contexts[0])

{'3': 'Ночной горшок', '1': 'Округлый глиняный сосуд для приготовления пищи (печной горшок)', '2': 'Расширяющийся кверху сосуд с отверстием в дне (цветочный горшок)'}
(('телевизор, - ковер, , - музыкальный центр, - стол, - аквариум, - 3 шкафа, - цветы в', ' горшках', ', - мелкие аксессуары.'), '2')


In [44]:
import pymystem3
import re
MyStem = pymystem3.Mystem()

In [67]:
word_re = re.compile('\w+', re.U)
def normalize(ctx):
    left, _, right = ctx
    text = ' '.join([left, right]).strip()
    text = re.sub('\d', '2', text)
    return [w for w in MyStem.lemmatize(' '.join(word_re.findall(text)))
            if w not in utils.stopwords and w.strip()]
print(contexts[0][0])
print(normalize(contexts[0][0]))

('телевизор, - ковер, , - музыкальный центр, - стол, - аквариум, - 3 шкафа, - цветы в', ' горшках', ', - мелкие аксессуары.')
['телевизор', 'ковер', 'музыкальный', 'центр', 'стол', 'аквариум', '2', 'шкаф', 'цветок', 'мелкий', 'аксессуар']


In [60]:
import numpy as np
documents = [dictionary.doc2bow(normalize(ctx)) for ctx, _ in contexts]
gamma, _ = lda.inference(documents)
pred_topics = gamma.argmax(axis=1)
true_labels = np.array([int(ans) for _, ans in contexts])
print(pred_topics[:10])
print(true_labels[:10])

[3 2 2 2 2 5 2 2 0 5]
[2 2 2 2 2 3 2 1 2 2]


In [62]:
from sklearn.metrics import v_measure_score, adjusted_rand_score
print('ARI:', adjusted_rand_score(true_labels, pred_topics))
print('V score:', v_measure_score(true_labels, pred_topics))

ARI: 0.2723552642496695
V score: 0.225054708464
